In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [ ]:
dataset, dataset_info = tfds.load('malaria', with_info=True,
                                  as_supervised=True,
                                  shuffle_files = True,
                                  split=['train'])

In [ ]:
for data in dataset[0].take(1):
  print(data)


In [ ]:
def splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):
  DATASET_SIZE = len(dataset)

  train_dataset = dataset.take(int(TRAIN_RATIO*DATASET_SIZE))

  val_test_dataset = dataset.skip(int(TRAIN_RATIO*DATASET_SIZE))
  val_dataset = val_test_dataset.take(int(VAL_RATIO*DATASET_SIZE))

  test_dataset = val_test_dataset.skip(int(VAL_RATIO*DATASET_SIZE))
  return train_dataset, val_dataset, test_dataset

In [ ]:
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

train_dataset, val_dataset, test_dataset = splits(dataset[0], TRAIN_RATIO, VAL_RATIO, TEST_RATIO )
print(list(train_dataset.take(1).as_numpy_iterator()),
     list(val_dataset.take(1).as_numpy_iterator()), list(test_dataset.take(1).as_numpy_iterator()))

In [ ]:
train_dataset

DATASET VISUALIZATION

In [ ]:
for i, (image, label) in enumerate(train_dataset.take(16)):
  ax = plt.subplot(4, 4, i + 1)

  plt.imshow(image)
  plt.title(dataset_info.features['label'].int2str(label))
  plt.axis('off')

In [ ]:
dataset_info

DATA PREPROCESSING

In [ ]:
IM_SIZE=224
def resizing(image,label):
  return tf.image.resize(image,(IM_SIZE,IM_SIZE)),label


In [ ]:
train_dataset=train_dataset.map(resizing)

In [ ]:
for image,label in train_dataset.take(1):
  print(image,label)

In [ ]:
IM_SIZE=224
def resize_rescal(image,label):
  return tf.image.resize(image,(IM_SIZE,IM_SIZE))/255.0,label


In [ ]:
train_dataset=train_dataset.map(resize_rescal)
val_dataset=val_dataset.map(resize_rescal)
test_dataset=test_dataset.map(resize_rescal)

In [ ]:
train_dataset

In [ ]:
val_dataset

In [ ]:
for image,label in train_dataset.take(1):
  print(image,label)

In [ ]:
train_dataset=train_dataset.shuffle(buffer_size= 8,reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
train_dataset

In [ ]:
val_dataset=val_dataset.shuffle(buffer_size= 8,reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
val_dataset

BULDING CONVENTS WITH TENSORFLOW

MODEL CREATION

In [ ]:
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Flatten,InputLayer

In [ ]:
lenet_model=tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(IM_SIZE,IM_SIZE,3)),
        tf.keras.layers.Conv2D(filters=6,kernel_size=5,strides=(1,1),padding='valid',activation='sigmoid'),
        MaxPool2D(pool_size=2,strides=2),

        tf.keras.layers.Conv2D(filters=16,kernel_size=5,strides=(1,1),padding='valid',activation='sigmoid'),
        MaxPool2D(pool_size=2,strides=2),

        Flatten(),
        Dense(1000,activation="sigmoid"),
        Dense(10,activation="sigmoid"),
        Dense(1,activation="sigmoid"),
    ])
lenet_model.summary()

In [ ]:
import numpy as np


y_true=np.array([0.0,])
y_pred=np.array([0.02,])

bce=tf.keras.losses.BinaryCrossentropy(from_logits=False)
bce(y_true,y_pred)

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import RootMeanSquaredError

In [ ]:
lenet_model.compile(
    optimizer=Adam(learning_rate=0.1),
    loss=BinaryCrossentropy(),

)

In [ ]:
history=lenet_model.fit(train_dataset,validation_data=val_dataset,epochs=5,verbose=1)

In [ ]:
train_dataset

In [ ]:
val_dataset

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss','val_loss'])
plt.show()

Model Evaluation

In [ ]:
test_dataset=test_dataset.batch(1)

In [ ]:
lenet_model.evaluate(test_dataset)

In [ ]:
lenet_model.predict(test_dataset.take(1))[0][0]